In [1]:
import networkx as nx
import numpy as np
import qiskit_ibm_runtime.fake_provider as fk 
from qiskit.transpiler import Layout, CouplingMap

/Users/Quentin/Documents/applications/PythonIDE/path_env/qiskit_env/lib/python3.8/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


# Find best circuit adptating error circuit according to the subgraph

this note modifies the  function <u>find_best_cycle</u>. These modifications make it possible to take into account physical errors on the machine and to choose the cyclic sub-graph with the fewest errors in the case where several sub-graphs of the same length are proposed by the algorithm. To make this code functional, the input parameter <b>couplingMap</b> of <u>find_best_cycle</u> and <u>adaptive_ring_layout</u> must be replace by the <b>backend</b>. 

In [2]:
def minErrorCircuit(cycles, properties):
    """
    Return the circuit with the minimum error mitigation of the given backend 
    """ 
    erroPerCycle = []
    # list of error mitigation per circuit 
    for idx_cycle in  range(len(cycles)):
        selected_qubits = cycles[idx_cycle]
        erroPerCycle.append(sum(properties.readout_error(qubit) for qubit in selected_qubits)) 
    idxmin = np.argmin(erroPerCycle) # get the minimum error mitigation circuit 
    return cycles[idxmin]

def get_min_index(vector):
    """
    get the minimum length index of a graph list
    """
    sizeExact = []
    sizeExact.append([len(vector[i]) for i in range(len(vector))])
    min_val = min(sizeExact[0])
    indices = [i for i, val in enumerate(sizeExact[0]) if val == min_val]
    return indices
    

def find_best_cycle(backend, n: int):
    """
    Find exact cycle of length n, else nearest larger cycle.
    In this version, if the list of the minimum sizes cycles give more than 1,
    then choose the circuit with the minimum error mitation.
    """

    # get the coupling map of the backend
    coupling_map = CouplingMap(backend.configuration().coupling_map)

    # get the list of cycliq subgraph
    G = nx.Graph()
    G.add_edges_from(coupling_map.get_edges())
    cycles = list(nx.cycle_basis(G))
    
    exact  = [c for c in cycles if len(c) == n]
    larger = [c for c in cycles if len(c) > n]
    properties = backend.properties()
    if exact:
         
        indices = get_min_index(exact)               # get the index of the minimum circuit
        elements = [exact[i] for i in indices]       # get the subgraph with lenght n
        return minErrorCircuit(elements, properties) # return the minimum mitigation error subgraph

    if larger:
        indices = get_min_index(larger)              # Pick smallest larger cycle
        elements = [larger[i] for i in indices]      # get the subgraph with lenght n
        return minErrorCircuit(elements, properties) # return the minimum mitigation error subgraph
    return None


def adaptive_ring_layout(logical_qubits: int, backend):
    """
    Adaptive layout for ring circuits with ancilla if needed.
    """
    cycle = find_best_cycle(backend, logical_qubits)
    if cycle is None:
        raise ValueError("No valid cycles found in hardware coupling map.")

    required_qubits = len(cycle)
    ancilla = required_qubits - logical_qubits


    return required_qubits, ancilla#, layout

In [3]:
backend = fk.FakeAuckland()
for n in range(1,20):
    required_qubits, ancilla_count = adaptive_ring_layout(n, backend)
    cycle = find_best_cycle(backend, required_qubits)